In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
from causallearn.search.ScoreBased.GES import ges
from causallearn.search.ConstraintBased.PC import pc
import itertools
from sklearn.preprocessing import StandardScaler
from dagma.nonlinear import DagmaMLP, DagmaNonlinear
from dagma.linear import DagmaLinear
from castle.algorithms import NotearsNonlinear, DAG_GNN
os.environ["JAVA_HOME"] = "/homes/shahashka/lucid_cd/amazon-corretto-21.0.7.6.1-linux-x64/lib/"
import pytetrad.tools.TetradSearch as ts
import torch
import os
import pickle
import networkx as nx
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

/homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-25 11:23:00,209 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2025-04-25 11:23:00,349 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [10]:
dose = "B"
df = pd.read_csv(f"data/huvec/cd_matrix_d{dose}.csv")
df_gene_expression = df.drop(columns=["radiation"])
num_genes = len(df_gene_expression.columns)
gene_names = df_gene_expression.columns
df_gene_expression = StandardScaler().fit_transform(df_gene_expression)
df_gene_expression = pd.DataFrame(data=df_gene_expression)
df_learn = pd.DataFrame(data=df_gene_expression)
df_learn.columns = gene_names
df_learn['radiation'] = df['radiation']
 

In [11]:
print(df)

    WDR44  NSA2  PTGS2  IFT52  ZMIZ1  MAF  ENPP1  LOC105377627  DAPK3  MKNK2  \
0    1618  3404   2288   1550   4847  128    412           190   5910   3456   
1    2213  5139   1413   2219   9267  118    510            73   5513   8203   
2    1785  4566   1381   2008   5481  138    598           252   5579   3954   
3    1386  3888    539   1954   4642   70    404           172   5282   3925   
4    1487  2848   2276   1246   4042   68    286           188   4821   2950   
5    2009  4752   1800   2316   6384  218    600           379   6659   4280   
6    1530  3964    696   1934   4791   88    404           201   5233   4082   
7    1785  4651    728   2333   6220   28    470            97   4963   5609   
8    1434  4522    657   2040   5789   40    366            85   4479   5115   
9    1526  4413    546   2025   4734   30    576           108   4154   4506   
10   1466  4428    582   1982   4601   38    504           134   4066   4547   
11   2113  5586   1204   2490   6998   5

In [12]:
#device = torch.device("mps" if torch.mps.is_available() else "cpu")
def dagma_nonlinear_local_learn(subproblem, use_skel):
    skel, data = subproblem
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    d = data.shape[1]
    X = data.values
    X_tensor = torch.from_numpy(X).type(torch.double).to(device)
    eq_model = DagmaMLP(dims=[d, np.min([10, d-1]), 1], bias=True, dtype=torch.double) # create the model for the structural equations, in this case MLPs
    eq_model = eq_model.to(device)
    model = DagmaNonlinear(eq_model, dtype=torch.double) # create the model for DAG learning
    # adj = model.fit(X_tensor, lambda1=0.02, lambda2=0.005) # fit the model with L1 reg. (coeff. 0.02) and L2 reg. (coeff. 0.005)
    adj = model.fit(X_tensor, lambda1=0.001, lambda2=0.0001) # fit the model with L1 reg. (coeff. 0.02) and L2 reg. (coeff. 0.005)

    return adj

def ges_non_param_local_learn(subproblem, use_skel):
    skel, data = subproblem
    result = ges(data.values, maxP=10, node_names=data.columns, score_func="local_score_CV_general")
    return result
def ges_discrete_poisson_local_learn(subproblem, use_skel):
    skel, data = subproblem
    result = ges(data.values, maxP=10, node_names=data.columns, score_func="local_score_BDeu")
    return result
def pc_kci_local_learn(subproblem, use_skel):
    skel, data = subproblem
    result = pc(data.values, alpha=0.05, indep_test='kci')
    return result
def dag_gnn_local_learn(subproblem, use_skel):
    skel, data = subproblem
    d = data.shape[1]
    X = data.values
    model = DAG_GNN(device_type="gpu")
    model.learn(X)
    return model.causal_matrix
    
def notears_mlp_local_learn(subproblem, use_skel):
    skel, data = subproblem
    d = data.shape[1]
    X = data.values
    model = NotearsNonlinear(device_type="gpu")
    adj = model.learn(X)
    # model = NotearsMLP(dims=[d, 10, 1], bias=True)
    # #print(model.fc1_neg.weight.device, X.device)
    # adj = notears_nonlinear(model, X, lambda1=0.01, lambda2=0.01, device='gpu')
    return adj

### Global learning 

In [13]:
ground_truth = nx.read_gexf(f"./data/huvec/cd_subgraph_d{dose}.gexf" )
print(ground_truth)

DiGraph with 1590 nodes and 3729 edges


In [ ]:
from GENIE3 import GENIE3
# global_adj = ges_local_learn([superstructure, df], use_skel=True)
#global_adj = dagma_nonlinear_local_learn([None, df_learn], use_skel=False)
global_adj = dag_gnn_local_learn([None, df[list(ground_truth.nodes)+['radiation']]], use_skel=False)
#global_adj = notears_mlp_local_learn([None, df.iloc[:,0:1000]], use_skel=False)
#print(np.sum(global_adj!=0))
#global_adj = GENIE3(df[list(ground_truth.nodes)+['radiation']].values, nthreads=16)


2025-04-25 11:35:07,360 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:165] - INFO: GPU is available.


cuda


2025-04-25 11:35:33,841 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 0, epoch: 299, h_new: 0.29655004912729055
2025-04-25 11:40:25,120 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 1, epoch: 299, h_new: 0.12342199212071137
2025-04-25 11:40:51,633 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 2, epoch: 299, h_new: 0.12342199212071137
2025-04-25 11:41:18,069 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 3, epoch: 299, h_new: 0.12342199212071137
2025-04-25 11:41:44,480 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] -

In [ ]:
import networkx as nx
global_net = nx.from_numpy_array(global_adj, create_using=nx.DiGraph)
global_net = nx.relabel_nodes(global_net, dict(zip(np.arange(len(global_net.nodes)), list(ground_truth.nodes)+['radiation'])))
print(global_net)

DiGraph with 919 nodes and 545 edges


In [ ]:
true_positive = 0
for edge in ground_truth.edges:
    if global_net.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)
print(len(list(global_net.neighbors("radiation"))))
sorted_nodes = sorted(global_net.out_degree(), key=lambda x: x[1], reverse=True)

# Print nodes and their degree
print("Nodes sorted by degree:")
for node, degree in sorted_nodes:
    print(f"{node}: degree {degree}")

('GADD45A', 'EEF1A1')
1
245
Nodes sorted by degree:
radiation: degree 245
COL1A1: degree 18
BCL2A1: degree 11
NUPR1: degree 9
FOSB: degree 8
CDKN2A: degree 7
ADRA1D: degree 7
ADIRF-AS1: degree 6
GLI2: degree 5
DSC3: degree 4
SIX2: degree 4
CCNA1: degree 4
ACE: degree 3
FLRT3: degree 3
ESR2: degree 3
AMPH: degree 3
GRB14: degree 3
MYCN: degree 3
NTRK2: degree 3
ANGPTL4: degree 3
MAF: degree 3
ASS1: degree 3
C1QTNF6: degree 3
HMOX1: degree 3
BST2: degree 2
TUBA1A: degree 2
EFNB1: degree 2
CCND2: degree 2
SELL: degree 2
SMAD7: degree 2
TAGLN2: degree 2
HSPA2: degree 2
HDAC9: degree 2
ICAM1: degree 2
MT2A: degree 2
TUBB3: degree 2
CENPV: degree 2
BAG2: degree 2
DKK1: degree 2
MME: degree 2
ZSCAN20: degree 2
PCDH12: degree 2
GPR4: degree 2
TUBA4A: degree 2
TUBB2B: degree 2
PDLIM7: degree 2
CDKN1A: degree 1
GADD45A: degree 1
SGK1: degree 1
GYPC: degree 1
PROX1: degree 1
CDC6: degree 1
NOTCH3: degree 1
HERC5: degree 1
FBXO32: degree 1
CENPF: degree 1
RPS27A: degree 1
NTN4: degree 1
PIF1: degr

### Superstructure

In [19]:

# Correlation matrix with Permutation testing
corr_mat = df_gene_expression.corr('pearson', numeric_only=True).to_numpy()
print(np.min(np.abs(corr_mat)))
np.fill_diagonal(corr_mat, 0)
print(np.max(corr_mat), np.argmax(corr_mat))
random_corr_coef = []
for i in range(10):
    shuffled_array = df_gene_expression.values
    [np.random.shuffle(x) for x in shuffled_array]
    shuffled_final_data_set = pd.DataFrame(data=shuffled_array)
    shuffle_corr_mat = shuffled_final_data_set.corr('pearson', numeric_only=True).to_numpy()
    np.fill_diagonal(shuffle_corr_mat, 0)
    print(np.max(shuffle_corr_mat), np.argmax(shuffle_corr_mat))
    random_corr_coef.append(np.max(shuffle_corr_mat))
print(random_corr_coef)
ci_interval = st.t.interval(0.95, len(random_corr_coef)-1, 
                            loc=np.mean(random_corr_coef), 
                            scale=st.sem(random_corr_coef))
print(ci_interval)
cutoff = ci_interval[1]

corr_mat[corr_mat<=cutoff] = 0
corr_mat[corr_mat>cutoff] = 1

superstructure = corr_mat
print(f"Superstructure contains {np.sum(superstructure)} edges which is \
        {np.sum(superstructure)/(superstructure.shape[1]**2)} fraction of all possible edges")

nan
nan 5886
0.9931338554691231 2400661
0.9922982607314539 19666634
0.9885011722553623 18559022
0.9933796287091708 5287612
0.9899321810732494 3587886
0.9894394392309578 9951651


KeyboardInterrupt: 

In [9]:
# STRING PPI
ppi_df = pd.read_csv("./data/prior_knowledge/string_ppi_gene_names.csv", header=0)
string_undirected_edges = {(start, end) for start, end in zip(ppi_df["protein1"], ppi_df["protein2"])}
ppi_superstructure = np.zeros((num_genes, num_genes))
for i, j in itertools.product(np.arange(num_genes), np.arange(num_genes)):
    start = df.columns[i]
    end = df.columns[j]
    if (start, end) in string_undirected_edges:
        ppi_superstructure[i][j] = 1
        
print(f"Superstructure contains {np.sum(ppi_superstructure)} edges which is \
        {np.sum(ppi_superstructure)/(ppi_superstructure.shape[1]**2)} fraction of all possible edges")



Superstructure contains 298183.0 edges which is         0.008606812943491176 fraction of all possible edges


In [8]:
superstructure = ppi_superstructure

### Partition network

In [12]:
with open(f"./data/huvec/cd_partition_d{dose}_new.pickle", 'rb') as f:
    custom_partition = pickle.load(f)
print(custom_partition)
for i, comm in custom_partition.items():
    comm += ['radiation']

{0: ['TAPBPL', 'PDIA3', 'DKK3', 'ETS1', 'FOXP1', 'FLI1', 'SHC1', 'CBL', 'VAV2', 'CTSD', 'ACE', 'CXCR4', 'JUND', 'CD9', 'VAV3', 'E2F1', 'B2M', 'UBC', 'NFIC', 'ATF2', 'CDR1', 'ETS2', 'RANGAP1', 'ELF1', 'LIN54', 'BCOR', 'PLCG2', 'MICA', 'GADD45A', 'VCP', 'FYN', 'PIK3CB', 'HMBS', 'SEC24D', 'CD82', 'WDR5', 'SLC4A2', 'BRD4', 'STAT2', 'MED12', 'KLF4', 'LNPEP', 'HFE', 'LMNB1', 'ATP8A2', 'ADRA1D', 'ATP8A1', 'CANX', 'DLGAP5', 'KPNA2', 'UBB', 'IRF1', 'ERG', 'MAP1S', 'HDAC1', 'BHLHE40', 'SLC4A11', 'STAT5A', 'CALR', 'RAD21', 'ARG2', 'FCGRT', 'GYPC', 'MAF', 'MYC', 'NOTCH1', 'SLC4A8', 'MAZ', 'CDKN1A', 'IRF2', 'PTPRJ', 'PIK3R1', 'TPST1', 'PCNA', 'TFRC', 'HLA-A', 'NUSAP1', 'HNRNPC', 'SEC23A', 'AR', 'ICAM1', 'EGR1', 'ZNF263', 'PROCR'], 1: ['BAG2', 'ETS1', 'REPIN1', 'AGPAT2', 'ATAD3B', 'AXIN2', 'RCC1', 'UTP14A', 'RHOT1', 'PALD1', 'IGF1R', 'JUN', 'UBC', 'STK40', 'UBE2C', 'GPR4', 'TSPAN13', 'WRN', 'SREBF2', 'CLIP1', 'BCOR', 'PA2G4', 'RFC3', 'KIF23', 'N4BP3', 'SRSF3', 'FLCN', 'SSR2', 'RFC4', 'PLSCR1', 'EIF3

In [13]:
def partition_problem(partition: dict, structure: np.ndarray, data: pd.DataFrame):
    sub_problems = []
    k = list(partition.keys())
    k.sort()
    for i in k:
        sub_nodes = partition[i]
        sub_structure = np.ones((len(sub_nodes), len(sub_nodes)))
        sub_nodes = list(sub_nodes)
        sub_data = data[sub_nodes] 
        sub_problems.append((sub_structure, sub_data))
    return sub_problems

### Learning subgraphs

In [14]:
import functools
from concurrent.futures import ProcessPoolExecutor
from cd_v_partition.causal_discovery import ges_local_learn

superstructure = np.ones((num_genes+1, num_genes+1))
subproblems = partition_problem(custom_partition, superstructure, df)
# Setup thread level parallelism
# func_partial = functools.partial(
#     dagma_nonlinear_local_learn,
#     use_skel=False,
# )
# n_comms = len(ec_partition_small)
# nthreads = n_comms  # each thread handles one partition
results = []
# chunksize = max(1, n_comms // nthreads)
for subproblem in subproblems:
    #adj = ges_non_param_local_learn(subproblem, use_skel=False)
    #adj = pc_kci_local_learn(subproblem, use_skel=False)
   # adj = dagma_nonlinear_local_learn(subproblem, use_skel=False)
    #adj = ges_local_learn(subproblem, use_skel=False)
    print(subproblem[1].columns)
    #adj = notears_mlp_local_learn(subproblem, use_skel=False)
    adj = ges_discrete_poisson_local_learn(subproblem, use_skel=False)
    print(np.sum(adj!=0))
    results.append(adj)
# with ProcessPoolExecutor(max_workers=nthreads) as executor:
#     for result in executor.map(func_partial, subproblems, chunksize=chunksize):
#         results.append(result)

Index(['TAPBPL', 'PDIA3', 'DKK3', 'ETS1', 'FOXP1', 'FLI1', 'SHC1', 'CBL',
       'VAV2', 'CTSD', 'ACE', 'CXCR4', 'JUND', 'CD9', 'VAV3', 'E2F1', 'B2M',
       'UBC', 'NFIC', 'ATF2', 'CDR1', 'ETS2', 'RANGAP1', 'ELF1', 'LIN54',
       'BCOR', 'PLCG2', 'MICA', 'GADD45A', 'VCP', 'FYN', 'PIK3CB', 'HMBS',
       'SEC24D', 'CD82', 'WDR5', 'SLC4A2', 'BRD4', 'STAT2', 'MED12', 'KLF4',
       'LNPEP', 'HFE', 'LMNB1', 'ATP8A2', 'ADRA1D', 'ATP8A1', 'CANX', 'DLGAP5',
       'KPNA2', 'UBB', 'IRF1', 'ERG', 'MAP1S', 'HDAC1', 'BHLHE40', 'SLC4A11',
       'STAT5A', 'CALR', 'RAD21', 'ARG2', 'FCGRT', 'GYPC', 'MAF', 'MYC',
       'NOTCH1', 'SLC4A8', 'MAZ', 'CDKN1A', 'IRF2', 'PTPRJ', 'PIK3R1', 'TPST1',
       'PCNA', 'TFRC', 'HLA-A', 'NUSAP1', 'HNRNPC', 'SEC23A', 'AR', 'ICAM1',
       'EGR1', 'ZNF263', 'PROCR', 'radiation'],
      dtype='object')


/homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/causallearn/search/ScoreBased/GES.py:45: UserWarning: The number of features is much larger than the sample size!
  warnings.warn("The number of features is much larger than the sample size!")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_56456/3608095113.py", line 23, in <cell line: 16>
    adj = ges_discrete_poisson_local_learn(subproblem, use_skel=False)
  File "/tmp/ipykernel_56456/1412092906.py", line 22, in ges_discrete_poisson_local_learn
    result = ges(data.values, maxP=10, node_names=data.columns, score_func="local_score_BDeu")
  File "/homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/causallearn/search/ScoreBased/GES.py", line 232, in ges
    Idx = find_subset_include(
  File "/homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/causallearn/utils/GESUtils.py", line -1, in find_subset_include
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/homes/shahashka/

In [50]:
from cd_v_partition.fusion import screen_projections, fusion
num_edges = 0
for r in results:
    print(r.shape)
    num_edges += np.sum(r != 0)
print(num_edges)
global_net_non_param = screen_projections(superstructure, custom_partition, results, ss_subset=False, finite_lim=False, data=df_learn.to_numpy())
print(len(global_net_non_param.edges()))
print(len(global_net_non_param.nodes()))

(726, 726)
(93, 93)
(892, 892)
(82, 82)
(209, 209)
(110, 110)
2767
1742
1488


In [ ]:
import networkx as nx
from netgraph import Graph
import matplotlib.pyplot as plt
ground_truth = nx.read_gexf(f"./data/huvec/cd_subgraph_d{dose}.gexf" )
print(ground_truth)
true_positive = 0
for edge in ground_truth.edges:
    if  global_net_non_param.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)

DiGraph with 1487 nodes and 3384 edges
('EGR2', 'POLR2L')
('EGR2', 'TEAD1')
('EGR2', 'POLR2B')
('EGR2', 'POLR2D')
('EGR2', 'WWTR1')
('EGR2', 'SCD5')
('EGR2', 'TTC3')
('EGR2', 'MICU1')
('EGR2', 'TMEM87B')
('EGR2', 'NUP188')
('EGR2', 'RBM39')
('EGR2', 'DONSON')
('EGR2', 'SMPD1')
('EGR2', 'FAF1')
('EGR2', 'ATP6V1G1')
('EGR2', 'LTBP3')
('EGR2', 'NPLOC4')
('EGR2', 'CCNI')
('EGR2', 'TMX4')
('EGR2', 'SLFN11')
('EGR2', 'SH3TC1')
('EGR2', 'BZW2')
('EGR2', 'PNRC1')
('EGR2', 'CAPN1')
('VAV3', 'B2M')
25


In [52]:
radbio_df = pd.read_csv("./data/prior_knowledge/radiation_edges.txt")
radbio_net = nx.DiGraph({edge for edge in zip(radbio_df['node1'], radbio_df['node2'])})
true_positive=0
for edge in radbio_net.edges:
    if  global_net_non_param.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)

0


In [53]:
nx.write_gexf(global_net_non_param, f"./data/huvec/cd_dag_gnn_d{dose}.gexf")

In [54]:
len(list(global_net_non_param.neighbors("radiation")))

135

In [55]:
sorted_nodes = sorted(global_net_non_param.out_degree(), key=lambda x: x[1], reverse=True)

# Print nodes and their degree
print("Nodes sorted by degree:")
for node, degree in sorted_nodes:
    print(f"{node}: degree {degree}")

Nodes sorted by degree:
FOXS1: degree 176
radiation: degree 135
EGR2: degree 129
CRYAB: degree 88
ADRA1D: degree 45
BCL2A1: degree 35
COL1A1: degree 33
CD38: degree 31
KLF4: degree 30
NUPR1: degree 28
MAFB: degree 26
IGF2: degree 25
ATP8A2: degree 23
CEP19: degree 21
SMAD6: degree 19
HFE: degree 16
MAF: degree 15
ICAM1: degree 15
FOSB: degree 14
GYPC: degree 13
WDR27: degree 13
KCNMA1: degree 12
VAV3: degree 12
TPST1: degree 12
PTPN7: degree 12
PPP1R14A: degree 10
GLI2: degree 10
BDKRB2: degree 10
SMAD7: degree 9
ACE: degree 9
TMEM184A: degree 9
BCL2L11: degree 9
NFATC2: degree 9
PABPC1L: degree 8
MAP3K14: degree 8
LRRC27: degree 8
TXNIP: degree 7
TRPC6: degree 7
CCNA1: degree 7
ATP8A1: degree 7
PTPN22: degree 6
EGR1: degree 6
PTGS2: degree 6
PBX3: degree 6
SLC4A11: degree 6
PTPRJ: degree 6
CCND2: degree 6
CDKN2C: degree 6
SETBP1: degree 5
VCAN: degree 5
ATF3: degree 5
LIN54: degree 5
SLC4A8: degree 5
ZNF263: degree 5
SIX2: degree 5
STAG3: degree 5
GADD45B: degree 5
LMO2: degree 5
GATA